In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import optimizers
from keras.utils.generic_utils import get_custom_objects
from keras.layers import GlobalMaxPool2D, Dropout, Dense, Activation, BatchNormalization
from keras.models import Model
import numpy as np
from keras.backend import sigmoid
from efficientnet.keras import EfficientNetB4 as NetB4
from efficientnet.keras import EfficientNetB4 as NetB5
from efficientnet.keras import EfficientNetB4 as NetB6
from efficientnet.keras import EfficientNetB4 as NetB7

In [ ]:
import cv2
import numpy as np
import os
import argparse
import keras
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import time
from mtcnn import MTCNN

In [ ]:
pip install mtcnn

## set parameters

In [ ]:
IMAGE_SIZE = 56
BATCH_SIZE = 256
WEIGHT_INIT = 0.08
NUM_AGE_CLASSES = 3
NUM_GENDER_CLASSES = 2
DROPOUT_RATE = 0.2
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
NUM_EPOCHS = 40
DECAY_LR_RATE = 0.9
NUM_AGE_CLASSES=3
min_score = 1.0

## lode the data

In [ ]:
import keras
import os
import numpy as np

class Datasets(object):
    def __init__(self,data_name):
        self.data_name=data_name
        self.datasets = self.getData(self)
        self.final_data = []
        self.convert_data_format()


    def gen(self):
        # np.random.shuffle(self.final_data)

        images = []
        age_labels = []
        gender_labels = []
        race_labels=[]

        for i in range(len(self.final_data)):
            image, age, gender,race = self.final_data[i]
            images.append(image)
            age_labels.append(age)
            gender_labels.append(gender)
            race_labels.append(race)
    
        age_labels = keras.utils.to_categorical(age_labels, num_classes=NUM_AGE_CLASSES)
        gender_labels = keras.utils.to_categorical(gender_labels, num_classes=2)
        race_labels = keras.utils.to_categorical(race_labels, num_classes=5)

        return images, age_labels, gender_labels, race_labels

    @staticmethod
    def getData(self):
        print('Loading age image...')
        # data_3
        if self.data_name=='data_3_utk':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_3_utk.npy'), allow_pickle=True) 
        if self.data_name=='data_5_utk':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_5_utk.npy'), allow_pickle=True) 
        if self.data_name=='data_10_utk':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_10_utk.npy'), allow_pickle=True)
        np.random.shuffle(data)
        all_data = []
        # random select 20000 data
        for i in range(20000):  ### number of samples
            all_data.append(data[i])
        print('Number of age data:', str(len(all_data)))
        return all_data

    def convert_data_format(self):
        # Age datasets:
        for i in range(len(self.datasets)):
            image = self.datasets[i][0] / 255.0
            age_labels = self.datasets[i][1]
            gender_labels = self.datasets[i][2]
            race_labels= self.datasets[i][3]

            self.final_data.append((image, age_labels, gender_labels,race_labels))

## model

In [ ]:
import pandas as pd
class SwishActivation(Activation):
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta=1):
    return x * sigmoid(beta * x)

class Efficient_Net(object):
    def __init__(self, model_name, data_name, trainable=True):
        self.trainable = trainable
        self.model_name=model_name
        self.data_name=data_name
        if self.trainable: 
            self.train_data = Datasets(data_name=self.data_name)
        self.model = self.build_model(self.model_name)

        # Compile the model
        losses = {
            "age_output": "categorical_crossentropy",
            "gender_output": "categorical_crossentropy",
             "race_output": "categorical_crossentropy"

        }

        opt = optimizers.Adam(1e-3)
        self.model.compile(loss=losses, optimizer=opt, metrics=['acc'])

        # Train the part you added
        if self.trainable:
            self.model.summary()

    @staticmethod
    def build_age_branch(x):
        # Output age branch
        predictions_age = Dense(NUM_AGE_CLASSES, activation="softmax", name='age_output')(x)

        return predictions_age

    @staticmethod
    def build_gender_branch(x):
        # Output gender branch
        predictions_gender = Dense(2, activation="softmax", name='gender_output')(x)

        return predictions_gender


    def build_race_branch(self,x):
        # Output race branch
        predictions_race = Dense(5, activation="softmax", name='race_output')(x)

        return predictions_race

    def build_model(self,model_name):
        get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

        # Model
        if model_name=='age_gender_race_B4':
          model = NetB4(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='age_gender_race_B5':
          model = NetB5(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='age_gender_race_B6':
          model = NetB6(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='age_gender_race_B7':
          model = NetB7(weights='imagenet', include_top=False, input_shape=input_shape)
        

        # Adding 2 fully-connected layers to B4.
        x = model.output
        x = BatchNormalization()(x)
        x = GlobalMaxPool2D(name='gap1')(x)
        x = Dropout(DROPOUT_RATE, name='dropout1')(x)

        # Output layer
        predictions_age = self.build_age_branch(x)
        predictions_gender = self.build_gender_branch(x)
        predictions_race = self.build_race_branch(x)
        model_final = Model(inputs=model.input, outputs=[predictions_age, predictions_gender,predictions_race])

        return model_final

    def train(self):
        # reduce learning rate
        reduce_lr = ReduceLROnPlateau(monitor='val_age_output_acc', factor=DECAY_LR_RATE, patience=5, verbose=1, )
        # Model Checkpoint
        # cpt_save = ModelCheckpoint( '/content/drive/MyDrive/3033proj/weights/weight_{}.h5'.format(model_name+data_name), save_best_only=True, monitor='val_age_output_acc', mode='max')
        # print("Training......")
        trainX, trainAgeY, trainGenderY ,trainRaceY= self.train_data.gen()
        trainX = np.array(trainX)



        history=self.model.fit(trainX, {"age_output": trainAgeY, "gender_output": trainGenderY, "race_output": trainRaceY}, validation_split=0.2,
                       callbacks=reduce_lr, verbose=1, epochs=40, shuffle=True,
                       batch_size=256)
        
        dir ='/content/drive/MyDrive/3033proj/model_log/'
        log_path = dir+self.model_name+self.data_name+'_2log.csv'
        hist = pd.DataFrame(history.history)
        hist.to_csv(log_path)

## inference

### 'age_gender_race_B6', 'data_3_utk'

In [ ]:
def draw_labels_and_boxes(img, boxes, result, margin):
    class_ids_age = np.argmax(result[0], axis=1)
    class_ids_gender = np.argmax(result[1], axis=1)
    class_ids_race = np.argmax(result[2], axis=1)

    if len(class_ids_age) <= 0:
        print('No age predicted')
        return None
    if len(class_ids_gender) <= 0:
        print('No gender predicted')
        return None
    if len(class_ids_race) <= 0:
        print('No race predicted')
        return None

    # Initialize color to perform each labels uniquely
    # colors = np.random.randint(0, 255, size=(num_age_label, 3), dtype='uint8')

    for i in range(len(class_ids_age)):
        # get the bounding box coordinates
        left, top, right, bottom = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
        width = right - left
        height = bottom - top
        img_h, img_w = img.shape[:2]

        x1 = max(int(left - margin * width), 0)
        y1 = max(int(top - margin * height), 0)
        x2 = min(int(right + margin * width), img_w - 1)
        y2 = min(int(bottom + margin * height), img_h - 1)
        # Get the unique color for this class
        # color = [int(c) for c in colors[class_ids_age[i]]]
        # Color red
        color = (0, 0, 255)

        # classify label according to result

        if class_ids_age[i] == 0:
            age = '1-26'
        elif class_ids_age[i] == 1:
            age = '27-52'
        elif class_ids_age[i] == 2:
            age = '53-80'
    

        if class_ids_gender[i] == 0:
            gender = 'Male'
        elif class_ids_gender[i] == 1:
            gender = 'Female'

        if class_ids_race[i] == 0:
            race = 'White'
        elif class_ids_race[i] == 1:
            race = 'Black'
        elif class_ids_race[i] == 2:
            race = 'Asian'
        elif class_ids_race[i] == 3:
            race = 'Indian'
        elif class_ids_race[i] == 4:
            race = 'Others'

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        text = 'Gender: {}, Age: {}, \nRace: {}'.format(gender, age,race)
        y0,dy=y1 - 15,10
        for i,txt in enumerate(text.split('\n')):
          y = y0+i*dy
          cv2.putText(img, txt, (x1 - 5, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, color, 1)
    return img


In [ ]:
NUM_AGE_CLASSES = 3
weight_path = '/content/drive/MyDrive/3033proj/weights/weight_age_gender_race_B6data_3_utk.h5'
img_path='/content/drive/MyDrive/3033proj/elder1.jpeg'

boxes = []

# Detector use by MTCNN
detector = MTCNN()

# Load image
image = cv2.imread(img_path)
img_h, img_w = image.shape[:2]

# Load weights
model = Efficient_Net('age_gender_race_B4', 'data_3_utk', trainable=True).model
model.load_weights(weight_path)

# Run detector through a image(frame)
start_time = time.time()
result = detector.detect_faces(image)
end_time = time.time()

num_detected_face = len(result)
if num_detected_face == 0:
    print('No detected face')
    exit()

faces = np.empty((num_detected_face, IMAGE_SIZE, IMAGE_SIZE, 3))
print("{}: detected {} faces on {}s".format(img_path, num_detected_face, end_time - start_time))

# crop faces
for i in range(len(result)):
    bounding_box = result[i]['box']
    keypoints = result[i]['keypoints']

    # coordinates of boxes
    left, top = bounding_box[0], bounding_box[1]
    right, bottom = bounding_box[0] + bounding_box[2], bounding_box[1] + bounding_box[3]

    # coordinates of cropped image
    x1_crop = max(int(left), 0)
    y1_crop = max(int(top), 0)
    x2_crop = int(right)
    y2_crop = int(bottom)

    cropped_face = image[y1_crop:y2_crop, x1_crop:x2_crop, :]
    face = cv2.resize(cropped_face, (IMAGE_SIZE, IMAGE_SIZE))
    faces[i, :, :, :] = face
    box = (x1_crop, y1_crop, x2_crop, y2_crop)
    boxes.append(box)

# predict
result = model.predict(faces / 255.0)

# Draw bounding boxes and labels on image
image = draw_labels_and_boxes(image, boxes, result, 0.001)

if image is None:
    exit()

image = cv2.resize(image, (img_w, img_h), cv2.INTER_AREA)
cv2.imwrite('selenagomez.jpg', image)
# cv2.imshow('img', image)
cv2_imshow(image)
if cv2.waitKey(0) & 0xFF == ord('q'):
    exit()

### age_gender_race_B4', 'data_5_utk'

In [ ]:
def draw_labels_and_boxes(img, boxes, result, margin):
    class_ids_age = np.argmax(result[0], axis=1)
    class_ids_gender = np.argmax(result[1], axis=1)
    class_ids_race = np.argmax(result[2], axis=1)

    if len(class_ids_age) <= 0:
        print('No age predicted')
        return None
    if len(class_ids_gender) <= 0:
        print('No gender predicted')
        return None
    if len(class_ids_race) <= 0:
        print('No race predicted')
        return None

    # Initialize color to perform each labels uniquely
    # colors = np.random.randint(0, 255, size=(num_age_label, 3), dtype='uint8')

    for i in range(len(class_ids_age)):
        # get the bounding box coordinates
        left, top, right, bottom = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
        width = right - left
        height = bottom - top
        img_h, img_w = img.shape[:2]

        x1 = max(int(left - margin * width), 0)
        y1 = max(int(top - margin * height), 0)
        x2 = min(int(right + margin * width), img_w - 1)
        y2 = min(int(bottom + margin * height), img_h - 1)
        # Get the unique color for this class
        # color = [int(c) for c in colors[class_ids_age[i]]]
        # Color red
        color = (0, 0, 255)

        # classify label according to result
        if class_ids_age[i] == 0:
            age = '1-26'
        elif class_ids_age[i] == 1:
            age = '27-52'
        elif class_ids_age[i] == 2:
            age = '53-80'
        elif class_ids_age[i] == 3:
            age = '40-55'
        elif class_ids_age[i] == 4:
            age = '56-80'


        if class_ids_gender[i] == 0:
            gender = 'Male'
        elif class_ids_gender[i] == 1:
            gender = 'Female'

        if class_ids_race[i] == 0:
            race = 'White'
        elif class_ids_race[i] == 1:
            race = 'Black'
        elif class_ids_race[i] == 2:
            race = 'Asian'
        elif class_ids_race[i] == 3:
            race = 'Indian'
        elif class_ids_race[i] == 4:
            race = 'Others'

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        text = 'Gender: {}, Age: {}, \nRace: {}'.format(gender, age,race)
        y0,dy=y1 - 15,10
        for i,txt in enumerate(text.split('\n')):
          y = y0+i*dy
          cv2.putText(img, txt, (x1 - 5, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, color, 1)
    return img


In [ ]:
NUM_AGE_CLASSES = 5
weight_path = '/content/drive/MyDrive/3033proj/weights/weight_age_gender_race_B4data_5_utk.h5'
img_path='/content/drive/MyDrive/3033proj/elder1.jpeg'

boxes = []

# Detector use by MTCNN
detector = MTCNN()

# Load image
image = cv2.imread(img_path)
img_h, img_w = image.shape[:2]

# Load weights
model = Efficient_Net('age_gender_race_B4', 'data_3_utk', trainable=True).model
model.load_weights(weight_path)

# Run detector through a image(frame)
start_time = time.time()
result = detector.detect_faces(image)
end_time = time.time()

num_detected_face = len(result)
if num_detected_face == 0:
    print('No detected face')
    exit()

faces = np.empty((num_detected_face, IMAGE_SIZE, IMAGE_SIZE, 3))
print("{}: detected {} faces on {}s".format(img_path, num_detected_face, end_time - start_time))

# crop faces
for i in range(len(result)):
    bounding_box = result[i]['box']
    keypoints = result[i]['keypoints']

    # coordinates of boxes
    left, top = bounding_box[0], bounding_box[1]
    right, bottom = bounding_box[0] + bounding_box[2], bounding_box[1] + bounding_box[3]

    # coordinates of cropped image
    x1_crop = max(int(left), 0)
    y1_crop = max(int(top), 0)
    x2_crop = int(right)
    y2_crop = int(bottom)

    cropped_face = image[y1_crop:y2_crop, x1_crop:x2_crop, :]
    face = cv2.resize(cropped_face, (IMAGE_SIZE, IMAGE_SIZE))
    faces[i, :, :, :] = face
    box = (x1_crop, y1_crop, x2_crop, y2_crop)
    boxes.append(box)

# predict
result = model.predict(faces / 255.0)

# Draw bounding boxes and labels on image
image = draw_labels_and_boxes(image, boxes, result, 0.001)

if image is None:
    exit()

image = cv2.resize(image, (img_w, img_h), cv2.INTER_AREA)
cv2.imwrite('selenagomez.jpg', image)
# cv2.imshow('img', image)
cv2_imshow(image)
if cv2.waitKey(0) & 0xFF == ord('q'):
    exit()

### age_gender_race_B4data_10_utk

In [ ]:
def draw_labels_and_boxes(img, boxes, result, margin):
    class_ids_age = np.argmax(result[0], axis=1)
    class_ids_gender = np.argmax(result[1], axis=1)
    class_ids_race = np.argmax(result[2], axis=1)

    if len(class_ids_age) <= 0:
        print('No age predicted')
        return None
    if len(class_ids_gender) <= 0:
        print('No gender predicted')
        return None
    if len(class_ids_race) <= 0:
        print('No race predicted')
        return None

    # Initialize color to perform each labels uniquely
    # colors = np.random.randint(0, 255, size=(num_age_label, 3), dtype='uint8')

    for i in range(len(class_ids_age)):
        # get the bounding box coordinates
        left, top, right, bottom = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
        width = right - left
        height = bottom - top
        img_h, img_w = img.shape[:2]

        x1 = max(int(left - margin * width), 0)
        y1 = max(int(top - margin * height), 0)
        x2 = min(int(right + margin * width), img_w - 1)
        y2 = min(int(bottom + margin * height), img_h - 1)
 
        color = (0, 0, 255)

        # classify label according to result

        if class_ids_age[i] == 0:
            age = '1-8'
        elif class_ids_age[i] == 1:
            age = '9-16'
        elif class_ids_age[i] == 2:
            age = '17-24'
        elif class_ids_age[i] == 3:
            age = '25-32'
        elif class_ids_age[i] == 4:
            age = '33-40'
        elif class_ids_age[i] == 5:
            age = '41-48'
        elif class_ids_age[i] == 6:
            age = '49-56'
        elif class_ids_age[i] == 7:
            age = '57-64'
        elif class_ids_age[i] == 8:
            age = '65-72'
        elif class_ids_age[i] == 9:
            age = '73-80'

        if class_ids_gender[i] == 0:
            gender = 'Male'
        elif class_ids_gender[i] == 1:
            gender = 'Female'

        if class_ids_race[i] == 0:
            race = 'White'
        elif class_ids_race[i] == 1:
            race = 'Black'
        elif class_ids_race[i] == 2:
            race = 'Asian'
        elif class_ids_race[i] == 3:
            race = 'Indian'
        elif class_ids_race[i] == 4:
            race = 'Others'

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        text = 'Gender: {}, Age: {}, \nRace: {}'.format(gender, age,race)
        y0,dy=y1 - 15,10
        for i,txt in enumerate(text.split('\n')):
          y = y0+i*dy
          cv2.putText(img, txt, (x1 - 5, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, color, 1)
    return img


In [ ]:
NUM_AGE_CLASSES = 10
weight_path = '/content/drive/MyDrive/3033proj/weights/weight_age_gender_race_B4data_10_utk.h5'
img_path='/content/drive/MyDrive/3033proj/elder1.jpeg'

boxes = []

# Detector use by MTCNN
detector = MTCNN()

# Load image
image = cv2.imread(img_path)
img_h, img_w = image.shape[:2]

# Load weights
model = Efficient_Net('age_gender_race_B4', 'data_3_utk', trainable=True).model
model.load_weights(weight_path)

# Run detector through a image(frame)
start_time = time.time()
result = detector.detect_faces(image)
end_time = time.time()

num_detected_face = len(result)
if num_detected_face == 0:
    print('No detected face')
    exit()

faces = np.empty((num_detected_face, IMAGE_SIZE, IMAGE_SIZE, 3))
print("{}: detected {} faces on {}s".format(img_path, num_detected_face, end_time - start_time))

# crop faces
for i in range(len(result)):
    bounding_box = result[i]['box']
    keypoints = result[i]['keypoints']

    # coordinates of boxes
    left, top = bounding_box[0], bounding_box[1]
    right, bottom = bounding_box[0] + bounding_box[2], bounding_box[1] + bounding_box[3]

    # coordinates of cropped image
    x1_crop = max(int(left), 0)
    y1_crop = max(int(top), 0)
    x2_crop = int(right)
    y2_crop = int(bottom)

    cropped_face = image[y1_crop:y2_crop, x1_crop:x2_crop, :]
    face = cv2.resize(cropped_face, (IMAGE_SIZE, IMAGE_SIZE))
    faces[i, :, :, :] = face
    box = (x1_crop, y1_crop, x2_crop, y2_crop)
    boxes.append(box)

# predict
result = model.predict(faces / 255.0)

# Draw bounding boxes and labels on image
image = draw_labels_and_boxes(image, boxes, result, 0.001)

if image is None:
    exit()

image = cv2.resize(image, (img_w, img_h), cv2.INTER_AREA)
cv2.imwrite('selenagomez.jpg', image)
# cv2.imshow('img', image)
cv2_imshow(image)
if cv2.waitKey(0) & 0xFF == ord('q'):
    exit()